In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from datetime import datetime

%matplotlib inline

log_dir = "logs"

def load_logs(log_dir):
    frames = []
    for file in os.listdir(log_dir):
        if file.endswith(".csv"):
            df = pd.read_csv(os.path.join(log_dir, file))
            frames.append(df)
    return pd.concat(frames, ignore_index=True)

df = load_logs(log_dir)
df["timestamp"] = pd.to_datetime(df["timestamp"])
df = df.sort_values("timestamp")

print("Dostępne czujniki:", df["sensor_id"].unique())

# Lista czujników i opisów
czujniki = {
    "T-001": "temperatura (°C)",
    "P-001": "ciśnienie (hPa)",
    "L-001": "światło (lx)",
    "H-001": "wilgotność (%)"
}

for sensor, opis in czujniki.items():
    df_temp = df[df["sensor_id"] == sensor].copy()
    df_temp["value"] = df_temp["value"].astype(float)

    średnia = df_temp["value"].mean()

    # Wygładzanie (jeśli wystarczająco danych)
    window = 51 if len(df_temp) > 100 else 5
    smoothed = savgol_filter(df_temp["value"], window_length=window, polyorder=3)

    # Wykres
    plt.figure(figsize=(12, 5))
    plt.plot(df_temp["timestamp"], df_temp["value"], alpha=0.5, label="Oryginalne")
    plt.plot(df_temp["timestamp"], smoothed, color='green', label="Wygładzone")
    plt.axhline(y=średnia, color="red", linestyle="--", label=f"Średnia = {średnia:.2f}")
    plt.title(f"Czujnik {sensor} – {opis}")
    plt.xlabel("Czas")
    plt.ylabel("Wartość")
    plt.legend()
    plt.grid()
    plt.show()
